**Постановка задачи для пункта б.)**

\begin{align}
    &\Delta u = -f(x, y), \\
    &f(x, y) = \frac{p}{k * h^2}, \quad \text{если } (x - x_0)^2 + (y - y_0)^2 \leq r^2, \\
    &f(x, y) = 0, \quad \text{иначе}, \\
    &0 < x < l, \quad 0 < y < l, \\
    &u(0, y) = 0, \\
    &u(x, 0) = 0, \\
    &u(l, y) = 0, \\
    &u(x, l) = 0.
\end{align}

In [54]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from scipy import sparse as sp
from scipy.sparse import linalg as la

In [55]:
def L_csr(n_x, n_y, h_x, h_y):
    k = np.arange(n_x * n_y, dtype=int).reshape((n_x, n_y))

    iB = k[:-1, :].flatten() # имеет нижних соседей
    iT = k[1:, ].flatten() # имеет верхних соседей

    iNB = k[-1, :].flatten()  # никаких соседей снизу
    iNT = k[0, :].flatten() # никаких соседей сверху

    iR = k[:, :-1].flatten() # имеет правых соседей
    iL = k[:, 1:].flatten() # имеет левых соседей

    iNR = k[:, -1].flatten() # никаких соседей справа
    iNL = k[:, 0].flatten() # никаких соседей слева

    iC = np.arange(n_x * n_y)

    R1, C1, V1 = iC, iC, np.full(iC.size, 2 / h_x ** 2 + 2 / h_y ** 2)

    R2, C2, V2 = iL, iR, np.full(iL.size, -1.0 / h_x ** 2)
    R3, C3, V3 = iR, iL, np.full(iR.size, -1.0 / h_x ** 2)

    R4, C4, V4 = iB, iT, np.full(iB.size, -1.0 / h_y ** 2)
    R5, C5, V5 = iT, iB, np.full(iT.size, -1.0 / h_y ** 2)

    R6, C6, V6 = iNR, iNR, np.full(iNR.size, 1.0 / h_x ** 2)
    R7, C7, V7 = iNL, iNL, np.full(iNL.size, 1.0 / h_x ** 2)

    R8, C8, V8 = iNB, iNB, np.full(iNB.size, 1.0 / h_y ** 2)
    R9, C9, V9 = iNT, iNT, np.full(iNT.size, 1.0 / h_y ** 2)

    row = np.concatenate((R1, R2, R3, R4, R5, R6, R7, R8, R9))
    col = np.concatenate((C1, C2, C3, C4, C5, C6, C7, C8, C9))
    val = np.concatenate((V1, V2, V3, V4, V5, V6, V7, V8, V9))

    return sp.csr_matrix((val, (row, col)))

In [56]:
h_0 = 0.01
l = 1
x_0 = 0.5
y_0 = 0.5
r = 0.1
p = 100 # мощность лазера
k = 385 # теплопроводность материала пластинки

n = int(np.round(l / h_0))

h = l / n

x = np.linspace(h / 2, l - h / 2, n)
y = np.linspace(h / 2, l - h / 2, n)

X, Y = np.meshgrid(x, y, indexing='ij')

In [57]:
def f(x, y, r):
    inside_circle = (x - x_0) ** 2 + (y - y_0) ** 2 <= r ** 2
    return np.where(inside_circle, (p / k) * (1 / h ** 2), 0)


def μ_L(y):
  return 0


def μ_R(y):
  return 0


def μ_B(x):
  return 0


def μ_T(x):
  return 0

In [58]:
def RHS(X, Y, h_x, h_y):
    res = f(X, Y, r)

    # i = 1
    res[0, :] += 2 * μ_L(Y[0, :]) / h_x ** 2

    # i = n_x
    res[-1, :] += 2 * μ_R(Y[-1, :]) / h_x ** 2

    # j = 1
    res[:, 0] += 2 * μ_B(X[:, 0]) / h_y ** 2

    # j = n_y
    res[:, -1] += 2 * μ_T(X[:, -1]) / h_y ** 2

    return res

In [59]:
L = L_csr(n, n, h, h)
F = RHS(X, Y, h, h).flatten()

In [60]:
u_num = la.spsolve(L, F).reshape((n, n))

In [61]:
fig = go.Figure()

fig.add_trace(go.Heatmap(z=u_num.T, colorscale='ice', x0=0, dx=l / u_num.shape[1], y0=0, dy=l / u_num.shape[0], colorbar=dict(title='u')))

fig.update_layout(
    width=500,
    height=500,
)

fig.update_xaxes(title_text='x')
fig.update_yaxes(title_text='y')

fig.show()